In [ ]:
%matplotlib inline
import os, sys
from datetime import datetime
sys.path.append("../model/")
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras.backend as K
import keras
import tensorflow as tf

from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

from unet import UNET

from train_utils import extract_contour, fill_mask, to_uint8
from train_utils import PlotCheckpoint, send_report_to_slack, set_directory, mean_iou, plot_sample_image, plot_bg_removal_sample_image
from train_utils import get_disc_batch, np_mean_iou, to_uint8
from data_utils import load_dataset, HumanSegGenerator
from data_utils import clahe_func, gray_func
from data_utils import rotation_func, rescaling_func, flip_func, random_crop_func, random_noise_func, gamma_func, color_gamma_func

In [ ]:
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))
###################################

First & Second Model
=====================

디폴트 실험. UNET을 기본으로 함. 

In [ ]:
def dice_coef_loss(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    return 1-(2. * intersection + 1e-5) / (union + 1e-5)
    
def bcf(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

def bcf_dice_loss(loss_d):
    def merge_loss(y_true, y_pred):
        dice_loss = dice_coef_loss(y_true, y_pred)
        bcf_loss = bcf(y_true, y_pred)
        return loss_d * dice_loss + (1. - loss_d) * bcf_loss
    return merge_loss

def dice_coef(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    return (2. * intersection + 1e-5) / (union + 1e-5)

def jaccard_distance(y_true, y_pred, smooth=100):
    """Jaccard distance for semantic segmentation, also known as the intersection-over-union loss.
    This loss is useful when you have unbalanced numbers of pixels within an image
    because it gives all classes equal weight. However, it is not the defacto
    standard for image segmentation.
    For example, assume you are trying to predict if each pixel is cat, dog, or background.
    You have 80% background pixels, 10% dog, and 10% cat. If the model predicts 100% background
    should it be be 80% right (as with categorical cross entropy) or 30% (with this loss)?
    The loss has been modified to have a smooth gradient as it converges on zero.
    This has been shifted so it converges on 0 and is smoothed to avoid exploding
    or disappearing gradient.
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    # References
    Csurka, Gabriela & Larlus, Diane & Perronnin, Florent. (2013).
    What is a good evaluation measure for semantic segmentation?.
    IEEE Trans. Pattern Anal. Mach. Intell.. 26. . 10.5244/C.27.32.
    https://en.wikipedia.org/wiki/Jaccard_index
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
model_name = "unet"
dataset_name = 'train_refined'
h5_path = "../data/baidu-segmentation.h5"

batch_size = 1
if batch_size == 1:
    instance = True
else:
    instance = False
    
nb_epoch = 200

img_dim = (256,256,3)
output_dim = (256,256,1)

nb_filter = 16
depth = 4
activation= 'LeakyReLU'
bn = True
lr = 0.0002
beta_1 = 0.5
loss_function = 'binary_crossentropy'
sigmoid = True
drop_rate = 0.
bg_removal = False

title = "17th trial - loss Change"
description = '''
Binary Cross Entropy
'''

In [ ]:
# create data generator
trainset = load_dataset(dataset_name=dataset_name,h5_path=h5_path)

epoch_size = len(trainset)
train_steps = epoch_size // batch_size

traingen = HumanSegGenerator(trainset,img_dim,
                             batch_size=batch_size,
                             sigmoid=sigmoid,
                             bg_removal=bg_removal,
                             aug_funcs=[rotation_func(), 
                                        rescaling_func(),
                                        flip_func(), 
                                        random_crop_func(),
                                        random_noise_func(),
                                        gamma_func(),
                                        color_gamma_func()],
                             prep_funcs=[])

testset = load_dataset(dataset_name='test',h5_path=h5_path)
test_images, test_profiles = next(HumanSegGenerator(testset,img_dim,
                                                    batch_size=len(testset),
                                                    sigmoid=sigmoid,
                                                    bg_removal=bg_removal,
                                                    prep_funcs=[]))

# create the model
unet = UNET(img_dim=img_dim,
            depth=depth,
            nb_filter=nb_filter,
            output_dim=output_dim,
            bn=bn,
            instance=instance,
            sigmoid=sigmoid)

In [ ]:
# set the model directory to save 
weights_dir, model_dir, sample_dir = set_directory("../results",model_name)

# plot the model arch image
model_path = os.path.join(model_dir, "unet.png")
plot_model(unet, to_file=model_path, show_shapes=True)

# serialize model to JSON
model_json = unet.to_json()
with open(os.path.join(model_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)

# set callback function
tqdm = TQDMNotebookCallback()
modelcheckpoint = ModelCheckpoint(os.path.join(weights_dir,
                                               "model_{val_mean_iou:.2f}.h5"),
                                  monitor='val_mean_iou',
                                  mode='max',
                                  save_weights_only=True,
                                  save_best_only=True,
                                  period=1)
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min',
                              patience=10)
plotcheckpoint = PlotCheckpoint(unet,test_images, sample_dir)
callbacks= [tqdm, modelcheckpoint, earlystopping, plotcheckpoint]

# set loss and optimizer
unet.compile(Adam(lr=lr,beta_1=beta_1),
            loss=loss_function,
            metrics=[mean_iou, dice_coef])

In [ ]:
hist = unet.fit_generator(traingen,
                           train_steps,
                           epochs=nb_epoch,
                           verbose=0, 
                           validation_data = (test_images, test_profiles),
                           callbacks=callbacks)

# Draw the results
df = pd.DataFrame({"train-loss":hist.history['loss'],
                   "val-loss":hist.history['val_loss'],
                   "train-mean_iou":hist.history['mean_iou'],
                   "val-mean_iou":hist.history['val_mean_iou']})
df.to_csv(os.path.join(model_dir,"training.csv"))
df.plot().figure.savefig(os.path.join(model_dir,"training.png"))

# serialize weights to HDF5
unet.save_weights(os.path.join(weights_dir,"model.h5"))
print("Saved model to disk")

# Send to results
best_idx = df['val-mean_iou'].sort_values(ascending=False).index[0]
best_iou = df['val-mean_iou'].sort_values(ascending=False).iloc[0]

description = description \
+ "\nmodel_path : {}".format(weights_dir[:-7])\
+ "\nbest_mean_iou : {}".format(best_iou)

image_paths = {
    'model' : os.path.join(model_dir,"unet.png"),
    'train-process' : os.path.join(model_dir,'training.png'),
    'train-process-csv' : os.path.join(model_dir,"training.csv"),
    'test image' : os.path.join(sample_dir,"{:02d}_epoch_sample.png".format(best_idx))
}
send_report_to_slack(title,description,image_paths)